# Clock Game Sync Word Finder

In [31]:
Numbers = [
    'dodici',
    'uno',
    'due',
    'tre',
    'quattro',
    'cinque',
    'sei',
    'sette',
    'otto',
    'nove',
    'dieci',
    'undici',
]

nNumbers = len(Numbers)

In [32]:
#Automaton for Italian Regions
Automaton = [
    [7, 11],
    [5, 3],
    [1, 3],
    [6, 0],
    [5, 1],
    [7, 6],
    [0, 7],
    [8, 11],
    [9, 5],
    [5, 4],
    [9, 8],
    [10, 8],
]

In [33]:
Automaton = [[len(n) % nNumbers, (len(n) + i) % nNumbers] for i, n in enumerate(Numbers)]
Automaton

[[6, 6],
 [3, 4],
 [3, 5],
 [3, 6],
 [7, 11],
 [6, 11],
 [3, 9],
 [5, 0],
 [4, 0],
 [4, 1],
 [5, 3],
 [6, 5]]

In [34]:
def Delta(state, move):
    return Automaton[state][move]

In [35]:
def Execute(word, starting_state):
    state = starting_state
    for c in word:
        state = Delta(state, int(c))
    return state

In [36]:
def hasKnownSyncWord(SyncWords, word):
    for w in SyncWords:
        if w in word:
            return True
    return False

In [37]:
maxLen = 15
iter = 2**maxLen
SyncWords = []

for n in range(iter):
    word = str(format(n + 2, 'b'))[1:]
    if not hasKnownSyncWord(SyncWords, word):
        outSet = set()

        for i in range(nNumbers):
            output = Execute(word, i)
            outSet.add(output)
            
        if(len(outSet) == 1):
            SyncWords.append(word)
            print(n, word.replace('0', 'A').replace('1', 'B') + ' ->', outSet)

30 AAAAA -> {3}
98 BAABAA -> {3}
252 BBBBBBA -> {6}
322 ABAAABAA -> {3}
494 BBBBAAAA -> {3}
530 AAAABABAA -> {3}
572 AAABBBBBA -> {6}
700 ABABBBBBA -> {6}
706 ABBAAABAA -> {3}
1012 BBBBBABBA -> {3}
1074 AAAABBABAA -> {3}
1134 AAABBBAAAA -> {3}
1306 ABAAABBBAA -> {3}
1309 ABAAABBBBB -> {11}
1330 ABAABBABAA -> {3}
1390 ABABBBAAAA -> {3}
1468 ABBABBBBBA -> {6}
1618 BAABABABAA -> {3}
2138 AAAABABBBAA -> {3}
2141 AAAABABBBBB -> {11}
2162 AAAABBBABAA -> {3}
2292 AAABBBBABBA -> {3}
2674 ABAABBBABAA -> {3}
2804 ABABBBBABBA -> {3}
2842 ABBAAABBBAA -> {3}
2845 ABBAAABBBBB -> {11}
2866 ABBAABBABAA -> {3}
2926 ABBABBBAAAA -> {3}
3250 BAABABBABAA -> {3}
3994 BBBBAABBBAA -> {3}
3997 BBBBAABBBBB -> {11}
4018 BBBBABBABAA -> {3}
4314 AAAABBABBBAA -> {3}
4317 AAAABBABBBBB -> {11}
5202 ABAAABABABAA -> {3}
5226 ABAAABBABBAA -> {3}
5338 ABAABBABBBAA -> {3}
5341 ABAABBABBBBB -> {11}
5362 ABAABBBBABAA -> {3}
5746 ABBAABBBABAA -> {3}
5876 ABBABBBBABBA -> {3}
6490 BAABABABBBAA -> {3}
6493 BAABABABBBBB -> {11}


In [39]:
def Step(State, move):
    res = set()
    for s in State:
        res.add(Automaton[s][move])
    return tuple(sorted(res))

def Neighbors(State):
    output = []
    for m in range(len(Automaton[0])):
        output.append(Step(State, m))
    return output

def minOpen(Open, FScores):
    minScore = FScores[Open[0]]
    out = Open[0]
    for o in Open[1:]:
        score = FScores[o]
        if score < minScore:
            minScore = score
            out = o
    return out

def BuildPath(CameFrom, node):
    if node in CameFrom:
        return BuildPath(CameFrom, CameFrom[node][1]) + str(CameFrom[node][0])
    else:
        return ""


In [40]:
def H_Dis(state):
    return len(state)

def G_dis(state1, state2):
    return 1

In [48]:
def ShortestWord():
    InitialState = tuple([_ for _ in range(nNumbers)])
    steps = 0

    Open = [InitialState]
    Closed = []

    GScores = {InitialState: 0}
    HScores = {InitialState: H_Dis(InitialState)}
    FScores = {InitialState: HScores[InitialState]}

    cameFrom = {}

    while len(Open) != 0:
        current = minOpen(Open, FScores)
        steps += 1

        if len(current) == 1:
            return BuildPath(cameFrom, current), current, steps
            
        Open.remove(current)
        Closed.append(current)

        for i, n in enumerate(Neighbors(current)):
            if n in Closed:
                continue

            tentativeGScore = GScores[current] + G_dis(current, n)

            if not n in Open:
                Open.append(n)
                betterTentative = True
            elif tentativeGScore < GScores[n]:
                betterTentative = True
            else:
                betterTentative = False

            if betterTentative:
                cameFrom[n] = (i, current)

                GScores[n] = tentativeGScore
                HScores[n] = H_Dis(n)
                FScores[n] = GScores[n] + HScores[n]
    return 'Failure'

word, end, steps = ShortestWord()
word = word.replace('0', 'A').replace('1', 'B')
print(steps, word, '->', end)


9 AAAAA -> (3,)
